In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import tensorflow as tf
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

<h2>Extract MNIST data</h2>

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
#get mnist data, with one_hot encoding
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
#suppress warnings
tf.logging.set_verbosity(old_v)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
#you can print out the label for 1st figure
print(mnist.train.labels[0])

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [4]:
num_train = mnist.train.num_examples #55,000
num_validation = mnist.validation.num_examples #5000
num_test = mnist.test.num_examples #10,000

<h2>Set hyperparameters</h2>

In [5]:
#learning rate
lr = 0.01
#number of traning steps
num_steps =500
#number of batch_size
batch_size = 128

#network parameters
n_hidden_1 = 300
n_hidden_2 = 100
num_input = 784
num_classes = 10

In [51]:
tf.reset_default_graph()

In [52]:
#tf graph input
X = tf.placeholder(tf.float32,[None,num_input],name='X')
Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')


In [53]:
#define a neural net model
def neural_net(x):
    layer_1_out = tf.layers.dense(x,n_hidden_1,activation=None,kernel_initializer=tf.random_normal_initializer())
    layer_2_out = tf.layers.dense(layer_1_out,n_hidden_2,activation=None,kernel_initializer=tf.random_normal_initializer())
    out = tf.layers.dense(layer_2_out,num_classes,activation=None,kernel_initializer=tf.random_normal_initializer())
    return out

In [54]:
#predicted labels
logits = neural_net(X)

#define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='loss')
#define optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

#compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))

#compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

#Initialize the variables
init = tf.global_variables_initializer()

In [55]:
for i in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
    print(i)   # i.name if you want just a name

<tf.Variable 'dense/kernel:0' shape=(784, 300) dtype=float32_ref>
<tf.Variable 'dense/bias:0' shape=(300,) dtype=float32_ref>
<tf.Variable 'dense_1/kernel:0' shape=(300, 100) dtype=float32_ref>
<tf.Variable 'dense_1/bias:0' shape=(100,) dtype=float32_ref>
<tf.Variable 'dense_2/kernel:0' shape=(100, 10) dtype=float32_ref>
<tf.Variable 'dense_2/bias:0' shape=(10,) dtype=float32_ref>


In [56]:
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(num_steps):
        #fetch batch
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        #run optimization
        sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
        if i % 100 ==0:
            acc = sess.run(accuracy,feed_dict={X:batch_x, Y:batch_y})
            print("step "+str(i)+", Accuracy= {:.3f}".format(acc))
    
    print("Training finished!")
    
    print("Testing ACcuracy:", sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))

step 0, Accuracy= 0.219
step 100, Accuracy= 0.891
step 200, Accuracy= 0.922
step 300, Accuracy= 0.883
step 400, Accuracy= 0.930
Training finished!
Testing ACcuracy: 0.8592


<h1>L2 Regurlarization example</h1>

In [72]:
tf.reset_default_graph()
#tf graph input
X = tf.placeholder(tf.float32,[None,num_input],name='X')
Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')

regularizer = tf.contrib.layers.l2_regularizer(0.1)

In [73]:
layer_1_out = tf.layers.dense(X,n_hidden_1,activation=None,kernel_initializer=tf.random_normal_initializer(),kernel_regularizer=regularizer)
layer_2_out = tf.layers.dense(layer_1_out,n_hidden_2,activation=None,kernel_initializer=tf.random_normal_initializer(),kernel_regularizer=regularizer)
logits = tf.layers.dense(layer_2_out,num_classes,activation=None,kernel_initializer=tf.random_normal_initializer(),kernel_regularizer=regularizer)

In [74]:
#define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='loss')
l2_loss = tf.losses.get_regularization_loss()
loss+=l2_loss
#define optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

#compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))

#compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

#Initialize the variables
init = tf.global_variables_initializer()

In [75]:
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(num_steps):
        #fetch batch
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        #run optimization
        sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
        if i % 100 ==0:
            acc = sess.run(accuracy,feed_dict={X:batch_x, Y:batch_y})
            print("step "+str(i)+", Accuracy= {:.3f}".format(acc))
    
    print("Training finished!")
    
    print("Testing ACcuracy:", sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))

step 0, Accuracy= 0.258
step 100, Accuracy= 0.914
step 200, Accuracy= 0.922
step 300, Accuracy= 0.953
step 400, Accuracy= 0.906
Training finished!
Testing ACcuracy: 0.8784


<h1>Dropout example</h1>

In [76]:
tf.reset_default_graph()
#tf graph input
X = tf.placeholder(tf.float32,[None,num_input],name='X')
Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')
training = tf.placeholder_with_default(False, shape=(),name='training')
dropout_rate = 0.5

In [77]:
#define a neural net model
X_drop = tf.layers.dropout(X,dropout_rate,training=training)
layer_1_out = tf.layers.dense(X_drop,n_hidden_1,activation=None,kernel_initializer=tf.random_normal_initializer())
layer_1_drop = tf.layers.dropout(layer_1_out,dropout_rate,training=training)
layer_2_out = tf.layers.dense(layer_1_drop,n_hidden_2,activation=None,kernel_initializer=tf.random_normal_initializer())
layer_2_drop = tf.layers.dropout(layer_2_out,dropout_rate,training=training)
out = tf.layers.dense(layer_2_drop,num_classes,activation=None,kernel_initializer=tf.random_normal_initializer())

In [78]:
#predicted labels
logits = out

#define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='loss')
#define optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

#compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))

#compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

#Initialize the variables
init = tf.global_variables_initializer()

In [79]:
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(num_steps):
        #fetch batch
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        #run optimization
        sess.run(train_op, feed_dict={X:batch_x, Y:batch_y,training:True})
        if i % 100 ==0:
            acc = sess.run(accuracy,feed_dict={X:batch_x, Y:batch_y})
            print("step "+str(i)+", Accuracy= {:.3f}".format(acc))
    
    print("Training finished!")
    
    print("Testing ACcuracy:", sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))

step 0, Accuracy= 0.406
step 100, Accuracy= 0.852
step 200, Accuracy= 0.875
step 300, Accuracy= 0.781
step 400, Accuracy= 0.852
Training finished!
Testing ACcuracy: 0.805


<h1>Batch Normalization example</h1>

In [34]:
tf.reset_default_graph()
#tf graph input
X = tf.placeholder(tf.float32,[None,num_input],name='X')
Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')
training = tf.placeholder_with_default(False, shape=(),name='training')

In [35]:
#define a neural net model
layer_1_out = tf.layers.dense(X,n_hidden_1,activation=None,kernel_initializer=tf.random_normal_initializer())
layer_1_bn = tf.layers.batch_normalization(layer_1_out,momentum=0.99,training=training)
layer_1_bn_act = tf.nn.relu(layer_1_bn)
layer_2_out = tf.layers.dense(layer_1_bn_act,n_hidden_2,activation='relu',kernel_initializer=tf.random_normal_initializer())
out = tf.layers.dense(layer_2_out,num_classes,activation=None,kernel_initializer=tf.random_normal_initializer())

In [36]:
#predicted labels
logits = out

#define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='loss')
#define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

#compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))

#compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

#Initialize the variables
init = tf.global_variables_initializer()

In [37]:
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(num_steps):
        #fetch batch
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        #run optimization
        sess.run(train_op, feed_dict={X:batch_x, Y:batch_y,training:True})
        if i % 100 ==0:
            acc = sess.run(accuracy,feed_dict={X:batch_x, Y:batch_y})
            print("step "+str(i)+", Accuracy= {:.3f}".format(acc))
    
    print("Training finished!")
    
    print("Testing ACcuracy:", sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))

step 0, Accuracy= 0.172
step 100, Accuracy= 0.562
step 200, Accuracy= 0.695
step 300, Accuracy= 0.781
step 400, Accuracy= 0.805
Training finished!
Testing ACcuracy: 0.767
